<a href="https://colab.research.google.com/github/hyona-yu/RL_RS_study/blob/master/CollaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import os
import ipywidgets
import traitlets
#####movielen latest small 사용 #####
#data_dir = './ml-latest-small/'
data_dir  = os.getcwd() + "/"
print(data_dir)
movies_csv = pd.read_csv(data_dir + "movies.csv")
links_csv = pd.read_csv(data_dir + "links.csv")
ratings_csv = pd.read_csv(data_dir + "ratings.csv")
tags_csv = pd.read_csv(data_dir + "tags.csv")

/content/


In [0]:
movies_csv.head()
#links_csv.head()
#tags_csv.head()
#ratings_csv.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
len(movies_csv)

9742

In [0]:
movie2rating = pd.merge(movies_csv, ratings_csv) #같은 column을 대상으로 합병
movie2rating = movie2rating.drop(columns = ['title', 'genres', 'timestamp'])
print(len(movie2rating))
movie2rating.head()

100836


,movieId,userId,rating
0,1,1,4.0
1,1,5,4.0
2,1,7,4.5
3,1,15,2.5
4,1,17,4.5


In [0]:
movies_csv['genres']

0       Adventure|Animation|Children|Comedy|Fantasy
1                        Adventure|Children|Fantasy
2                                    Comedy|Romance
3                              Comedy|Drama|Romance
4                                            Comedy
                           ...                     
9737                Action|Animation|Comedy|Fantasy
9738                       Animation|Comedy|Fantasy
9739                                          Drama
9740                               Action|Animation
9741                                         Comedy
Name: genres, Length: 9742, dtype: object

In [0]:
#https://www.kaggle.com/manisha1409raj/recommender-system-collaborative-filtering 참조
rating = ratings_csv.pivot_table(index = ['movieId'], columns = ['userId'], values ='rating') #pivot table로 데이터 재구조가능.
rating.fillna(0, inplace = True) #inplace = True를 해야 적용됨.
rating.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(movie2rating[:1000])
sim
sim_matrix = pd.DataFrame(sim)
sim_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.800132,0.731163,0.402123,0.481737,0.423547,0.436436,0.398124,0.346818,0.390129,0.329850,0.327056,0.356561,0.348372,0.304556,0.323391,0.341216,0.296393,0.291945,0.321257,0.313245,0.297694,0.295837,0.273616,0.304632,0.296459,0.244980,0.286673,0.267177,0.281971,0.269960,0.269582,0.279448,0.268495,0.286858,0.281089,0.274385,0.272948,0.262956,0.265289,...,0.345079,0.332495,0.296994,0.332212,0.326337,0.283741,0.313389,0.272563,0.290974,0.285058,0.287610,0.283790,0.287819,0.281008,0.279244,0.277166,0.280159,0.269203,0.277014,0.266777,0.267111,0.270453,0.261225,0.268254,0.253791,0.263959,0.266861,0.260600,0.261104,0.264125,0.257772,0.260072,0.257952,0.258436,0.257854,0.252237,0.259811,0.255440,0.254717,0.256632
1,0.800132,1.000000,0.994136,0.870755,0.911072,0.882244,0.888889,0.868779,0.840069,0.864429,0.830159,0.828508,0.845683,0.840982,0.815004,0.826348,0.836834,0.810017,0.807281,0.825071,0.820269,0.810823,0.809683,0.795854,0.815057,0.810064,0.777524,0.804024,0.791789,0.801098,0.793555,0.793315,0.799522,0.792628,0.804137,0.800548,0.796347,0.795442,0.789112,0.790597,...,0.813811,0.815709,0.794619,0.821406,0.819225,0.793079,0.813047,0.788652,0.801090,0.798616,0.801009,0.799166,0.801975,0.797789,0.796900,0.795839,0.797981,0.791116,0.796291,0.789861,0.790282,0.792671,0.786890,0.791430,0.782097,0.788728,0.790638,0.786698,0.787110,0.789064,0.784972,0.786518,0.785193,0.785572,0.785234,0.781604,0.786541,0.783817,0.783389,0.784649
2,0.731163,0.994136,1.000000,0.917935,0.950000,0.927421,0.932770,0.916665,0.893139,0.913342,0.884922,0.883547,0.897998,0.894110,0.872238,0.881851,0.890667,0.868036,0.865724,0.880842,0.876807,0.868789,0.867825,0.855993,0.872413,0.868171,0.840181,0.863028,0.852534,0.860541,0.854069,0.853865,0.859199,0.853279,0.863159,0.860090,0.856493,0.855721,0.850278,0.851565,...,0.859698,0.864902,0.847269,0.872081,0.870816,0.848645,0.866319,0.845978,0.856994,0.855453,0.857883,0.856587,0.859137,0.855604,0.854963,0.854191,0.856167,0.850323,0.854890,0.849415,0.849902,0.852120,0.847222,0.851159,0.843087,0.848862,0.850543,0.847200,0.847618,0.849314,0.845778,0.847160,0.846047,0.846425,0.846162,0.843025,0.847328,0.845054,0.844717,0.845827
3,0.402123,0.870755,0.917935,1.000000,0.995598,0.999577,0.999006,0.999825,0.998103,0.999414,0.996842,0.996604,0.998335,0.997878,0.994379,0.996135,0.997416,0.993438,0.992896,0.995833,0.995075,0.993487,0.993268,0.990488,0.994184,0.993301,0.986104,0.992137,0.989536,0.991522,0.989910,0.989854,0.991183,0.989693,0.992097,0.991374,0.990483,0.990279,0.988854,0.989182,...,0.965885,0.977768,0.974647,0.985621,0.986831,0.981897,0.987650,0.983684,0.987320,0.988034,0.989322,0.989444,0.990310,0.989480,0.989480,0.989458,0.990151,0.988637,0.990009,0.988569,0.988825,0.989573,0.988261,0.989385,0.987058,0.988764,0.989255,0.988342,0.988492,0.988977,0.987965,0.988383,0.988074,0.988199,0.988131,0.987213,0.988473,0.987828,0.987731,0.988053
4,0.481737,0.911072,0.950000,0.995598,1.000000,0.997779,0.998675,0.995584,0.988876,0.994816,0.986068,0.985577,0.990485,0.989242,0.981324,0.984991,0.988098,0.979653,0.978714,0.984621,0.983109,0.979969,0.979580,0.974610,0.981410,0.979721,0.967469,0.977611,0.973103,0.976567,0.973781,0.973692,0.975998,0.973435,0.977666,0.976377,0.974838,0.974504,0.972109,0.972682,...,0.949858,0.961658,0.955245,0.970484,0.971414,0.962465,0.971388,0.963855,0.969451,0.970004,0.971824,0.971813,0.973184,0.971753,0.971693,0.971601,0.972710,0.970211,0.972441,0.970086,0.970504,0.971744,0.969653,0.971470,0.967754,0.970477,0.971286,0.969842,0.970116,0.970902,0.969281,0.969975,0.969504,0.969741,0.969655,0.968206,0.970233,0.969278,0.969159,0.969693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [0]:
try : 
  user_inp = 'Jumanji (1995)'
  inp = movies_csv[movies_csv['title']==user_inp].index.tolist()
  inp = inp[0]
  movies_csv['similarity'] = sim_matrix.iloc[inp]
  print(movies_csv.head())
except:
  print("No")

   movieId  ... similarity
0        1  ...   0.800132
1        2  ...   1.000000
2        3  ...   0.994136
3        4  ...   0.870755
4        5  ...   0.911072

[5 rows x 4 columns]


In [0]:
#####Matrix Factorization 구현
# https://yamalab.tistory.com/92 참조
#R : original rating data matrix
#Nu : number of users
#Ni : number of items
R = rating.values #이러면 행렬로만 표현
num_item = len(movies_csv) #9742
num_user = ratings_csv['userId'].tolist()[-1] #610
nf = 50 #dimension of laten factor


In [0]:
print(R.shape)


(9724, 610)


(100, 610)

In [0]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)]) #np.where : 배열 안의 조건값에 해당하는거 다 포함해줌.

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))
        return self.get_complete_matrix()


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero() # 0이아닌값 index를 반환. 여기선 이차원배열이니깐 이런가보다.
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2) #제곱. 음수 없어주려고 그런듯?
        return np.sqrt(cost) / len(xi)


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

In [0]:
factorizer = MatrixFactorization(R[:100,:100], k=3, learning_rate=0.01, reg_param=0.01, epochs=300, verbose=True)
new_R  = factorizer.fit()
#factorizer.print_results()

Iteration: 10 ; cost = 0.0314
Iteration: 20 ; cost = 0.0272
Iteration: 30 ; cost = 0.0252
Iteration: 40 ; cost = 0.0239
Iteration: 50 ; cost = 0.0229
Iteration: 60 ; cost = 0.0220
Iteration: 70 ; cost = 0.0214
Iteration: 80 ; cost = 0.0208
Iteration: 90 ; cost = 0.0203
Iteration: 100 ; cost = 0.0199
Iteration: 110 ; cost = 0.0196
Iteration: 120 ; cost = 0.0192
Iteration: 130 ; cost = 0.0189
Iteration: 140 ; cost = 0.0187
Iteration: 150 ; cost = 0.0184
Iteration: 160 ; cost = 0.0182
Iteration: 170 ; cost = 0.0179
Iteration: 180 ; cost = 0.0177
Iteration: 190 ; cost = 0.0175
Iteration: 200 ; cost = 0.0173
Iteration: 210 ; cost = 0.0172
Iteration: 220 ; cost = 0.0170
Iteration: 230 ; cost = 0.0169
Iteration: 240 ; cost = 0.0167
Iteration: 250 ; cost = 0.0166
Iteration: 260 ; cost = 0.0165
Iteration: 270 ; cost = 0.0164
Iteration: 280 ; cost = 0.0163
Iteration: 290 ; cost = 0.0162
Iteration: 300 ; cost = 0.0161


In [0]:
print(new_R[:5,:5])
print(R[:5,:5])

[[4.29436717 4.82867811 1.80246125 1.0027213  3.71222304]
 [4.31826305 3.91412172 3.1342571  2.12912541 3.64636337]
 [3.81222571 2.86356107 2.52045444 1.76559182 3.36323015]
 [3.92912781 4.74959935 2.02872073 1.03841445 3.13441383]
 [3.74935103 3.8195622  1.32768369 0.84392464 3.38127534]]
[[4. 0. 0. 0. 4.]
 [0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
